In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from glob import glob

In [2]:
path = "/glade/work/eleanorm/cldlck_data/cldlck_polar/"

In [3]:
rundict = {
    'CNT': ['b.e11.B1850C5CN.f09_g16.005','b.e11.B1850C5CN.f09_g16.005_2xCO2'],
    'GCL': ['B1850C5CN.clock.9p.global','B1850C5CN.clock.9p.2xCO2'],
    'ACL': ['B1850C5CN.clock.9p.70-90N','B1850C5CN.clock.9p.70-90N.2xCO2']
}
configs = list(rundict.keys())

In [4]:
configs

['CNT', 'GCL', 'ACL']

In [5]:
def calc_nht_1x(config):
    compdeets = {}
    compdeets['cam'] = ['atm','tot']
    compdeets['pop'] = 'ocnnht'
    compdeets['cice'] = 'icenht'
    compnames = list(compdeets.keys())
    
    compvals = []
    for comp in compnames:
        co2name = glob(path+rundict[config][1]+"."+comp+"*heattransport.nc")
        htco2 = xr.open_dataset(co2name[0])
        pictrlname = glob(path+rundict[config][0]+"."+comp+"*heattransport.nc")
        ht = xr.open_dataset(pictrlname[0])
        if comp=='cam':
            for atm in compdeets['cam']:
                pival = ht[atm].loc[dict(lat=slice(69.5,70.5))].values[0]
                compvals.append(pival)
        else:
            pival = ht[compdeets[comp]].loc[dict(lat=slice(69.5,70.5))].values[0]
            compvals.append(pival)
    
    return compvals

In [6]:
def calc_nht_response(config):
    compdeets = {}
    compdeets['cam'] = ['atm','tot']
    compdeets['pop'] = 'ocnnht'
    compdeets['cice'] = 'icenht'
    compnames = list(compdeets.keys())
    
    compvals = []
    for comp in compnames:
        co2name = glob(path+rundict[config][1]+"."+comp+"*heattransport.nc")
        htco2 = xr.open_dataset(co2name[0])
        pictrlname = glob(path+rundict[config][0]+"."+comp+"*heattransport.nc")
        ht = xr.open_dataset(pictrlname[0])
        if comp=='cam':
            for atm in compdeets['cam']:
                co2val = htco2[atm].loc[dict(lat=slice(69.5,70.5))].values[0]
                pival = ht[atm].loc[dict(lat=slice(69.5,70.5))].values[0]
                compvals.append(co2val-pival)
        else:
            co2val = htco2[compdeets[comp]].loc[dict(lat=slice(69.5,70.5))].values[0]
            pival = ht[compdeets[comp]].loc[dict(lat=slice(69.5,70.5))].values[0]
            compvals.append(co2val-pival)
    
    return compvals

## Extract NHT in 1xCO2 simulation

In [9]:
df1x = {}
for config in configs:
    df1x[config] = calc_nht_1x(config)

df1x = pd.DataFrame.from_dict(df1x)
df1x.insert(0, 'comp', ['atm','total','ocean','ice'])
df1x.set_index('comp', inplace=True)

In [10]:
df1x

,CNT,GCL,ACL
comp,,,
atm,1.552443,1.557920,1.566005
total,1.766670,1.753875,1.773767
ocean,0.163620,0.162416,0.164584
ice,0.034534,0.026477,0.031708


In [12]:
neworder = ['atm','ocean','ice','total']
df1x=df1x.reindex(neworder)

In [13]:
df1x.to_csv('./nht_1xCO2_cldlck.csv',index=True)

## Extract NHT response to CO2 doubling

In [ ]:
df = {}
for config in configs:
    df[config] = calc_nht_response(config)

df = pd.DataFrame.from_dict(df)
df.insert(0, 'comp', ['atm','total','ocean','ice'])
df.set_index('comp', inplace=True)

In [ ]:
df

In [ ]:
neworder = ['atm','ocean','ice','total']
df=df.reindex(neworder)

In [ ]:
df.to_csv('./nht_response_2xCO2_cldlck.csv',index=True)